# BERTopic

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import df_to_list

In [2]:
import pandas as pd
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'adela'
df = df_caso(caso)

df = procesar_adela(df)
df = df[df['opt_left'] == 'Producir el alimento contra déficit vitamínico']

df = process_df(df,'comment','sel',verbose=True)

train_indices = pd.read_csv('../utils/splits/train_indices_{}.csv'.format(caso), header=None)[0].tolist()
test_indices = pd.read_csv('../utils/splits/test_indices_{}.csv'.format(caso), header=None)[0].tolist()
print("Partición train-test cargada")

df_train = df.iloc[train_indices]
df_test = df.iloc[test_indices]

26 rows found with non string elements for column comment (0.40%)
Deleting 685 columns for which max target value is over 7 (10.58%)
5761 available rows after processing
Partición train-test cargada


Guardando la partición train-test

In [3]:
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

docs_train = df_to_list(df_train,'comment',tokenizador)
docs_test = df_to_list(df_test, 'comment',tokenizador)

## BERTopic algorithm

In [4]:
from IPython.display import Image, display
display(Image(url='https://maartengr.github.io/BERTopic/algorithm/modularity.svg'))

In [5]:
%%capture
from bertopic import BERTopic

In [6]:
model = BERTopic(language="spanish",min_topic_size=10,nr_topics=25,calculate_probabilities=True)

In [7]:
%%time
topics, probs = model.fit_transform(docs_train)

CPU times: user 1min 9s, sys: 3.22 s, total: 1min 12s
Wall time: 32.9 s


In [8]:
model.get_topic_freq().head(5)

,Topic,Count
6,0,1050
4,-1,967
2,1,912
5,2,785
0,3,450


In [9]:
model.get_topic(0)[:10]

[('vitamina', 0.04430047448982247),
 ('déficit', 0.036808130746463844),
 ('vitamínico', 0.03222842031962001),
 ('tradiciones', 0.025239648354502663),
 ('alimento', 0.02521085487689096),
 ('importante', 0.022403852312724775),
 ('producir', 0.020919201832737676),
 ('vitaminas', 0.019797244211233026),
 ('deficit', 0.01929089916901348),
 ('pueblo', 0.01907947053004304)]

Topicos para algunos documentos

In [10]:
for i, t in enumerate(model.topics_[:10]):
    print('Tópico {}:\ttexto: {}'.format(t,docs_test[i]))

Tópico 3:	texto: Adela debería priorizar salud personas alguna creencia grupo pequeño personas
Tópico 3:	texto: Se debe priorizar respetar gente pueblo insistir cantidad gente beneficiaria fruta llegar acuerdo beneficie bandos problema
Tópico 4:	texto: A pesar ayudando porcentaje población método llevando cabo adecuado pues pasando llevar pueblo preguntando pesar fruto acostumbran cultivar mismo Este pueblo sintiéndose invadido pues arrebatando tradición
Tópico 1:	texto: El cultivar alimento tradiciones pueblo originario gran falta respeto parte integrantes proyecto eliminando cultura identidad pueblo provocando aún separación resto Chile sigue siendo detonante conflictos pueblos
Tópico 5:	texto: ser menor cabntidad producirian llegaria acuerdo facilmente pueblos originarios explicandoles realmente necesario debido si permiten uso fruto alimento-medcicina niños adultos mayores veran riesgo vida
Tópico -1:	texto: Desde perspectiva respetar dignidad integrantes pueblos originarios puesto

Visualización

In [11]:
model.visualize_topics()

Podemos relacionar tópicos con palabras en específico

In [12]:
target = 'integridad'

topic_num = 5

similar_topics, similarity = model.find_topics(target, top_n=topic_num)
similar_topics

[13, 10, 17, 11, 14]

In [13]:
similarity

[0.5201327165496956,
 0.33273560867202556,
 0.32604916603811096,
 0.3214980358176935,
 0.32021616389699237]

In [14]:
data = [[] for _ in range(topic_num)]
max_w = 5

for i, t in enumerate(similar_topics):
    data[i].append(similarity[i])
    words_top = model.get_topics(t)
    data[i] += [tup[0] for tup in model.get_topic(t)[:max_w]]

In [15]:
from utils.exploracion import print_table_md

print("Tópicos relevantes para concepto: {}".format(target))

headers = ['Probabilidad'] + ['Palabra {}'.format(i+1) for i in range(max_w)]

print_table_md(headers, data)

Tópicos relevantes para concepto: integridad


| Probabilidad | Palabra 1 | Palabra 2 | Palabra 3 | Palabra 4 | Palabra 5 |
| --- | --- | --- | --- | --- | --- |
| 0.5201327165496956 | postura | mantengo | misma | no | mitad |
| 0.33273560867202556 | identitarias | tradiciones | identidad | resguardar | pueblos |
| 0.32604916603811096 | mayor | personas | pueblo | acuerdo | llegar |
| 0.3214980358176935 | proyecto | considerar | acuerdo | si | llevar |
| 0.32021616389699237 | ambas | partes | acuerdo | llegar | dos |


Guardando el modelo

In [16]:
model.save("modelos/BT-{}.bertopic".format(caso))

Para cargar:

`my_model = BERTopic.load("modelos/BT-caso")`